In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis


In [2]:
from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main
from mol_opt.mol_opt import MolOpt

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

In [3]:
sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"

In [9]:
molopt, config = load_model(args.output_dir + "model_gcn2_34", MolOpt, args.device)
molopt, config

(MolOpt(
   (GCN): GCN(
     (W_message_i): Linear(in_features=100, out_features=50, bias=False)
     (W_message_h): Linear(in_features=50, out_features=50, bias=False)
     (W_message_o): Linear(in_features=143, out_features=50, bias=True)
     (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
     (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
     (dropout_gcn): Dropout(p=0.0, inplace=False)
     (dropout_ffn): Dropout(p=0.0, inplace=False)
   )
   (opt0): Linear(in_features=50, out_features=50, bias=True)
   (opt1): Linear(in_features=50, out_features=50, bias=True)
   (delta_loss): MSELoss()
 ),
 Namespace(agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dropout_ffn=0.0, dropout_gcn=0.0, ffn_activation='LeakyReLU', init_model='gcn2', linear_out=False, n_epochs=200, n_ffn_hidden=100, n_hidden=50, n_labels=1, n_layers=5, output_dir='mol_opt/output', pc_hidden=50))

In [10]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train", 48, True)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, True)

In [11]:
for i in val_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [13]:
x_embedding, x_delta_hat = molopt.forward(X)
y_embedding = molopt.encode(Y)

In [14]:
x_embedding + x_delta_hat

tensor([[-0.3318,  0.0026,  0.0829,  ...,  0.0947,  1.3627, -0.0060],
        [-0.3317,  0.0026,  0.0835,  ...,  0.0940,  1.3624, -0.0054],
        [-0.3315,  0.0024,  0.0871,  ...,  0.0892,  1.3614, -0.0014],
        ...,
        [-0.3317,  0.0026,  0.0838,  ...,  0.0936,  1.3623, -0.0049],
        [-0.3316,  0.0026,  0.0839,  ...,  0.0930,  1.3622, -0.0042],
        [-0.3316,  0.0026,  0.0833,  ...,  0.0943,  1.3624, -0.0057]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [16]:
y_embedding

tensor([[-0.3318,  0.0026,  0.0828,  ...,  0.0946,  1.3626, -0.0059],
        [-0.3316,  0.0025,  0.0844,  ...,  0.0922,  1.3619, -0.0033],
        [-0.3316,  0.0025,  0.0844,  ...,  0.0922,  1.3619, -0.0033],
        ...,
        [-0.3318,  0.0026,  0.0828,  ...,  0.0946,  1.3626, -0.0059],
        [-0.3316,  0.0026,  0.0832,  ...,  0.0942,  1.3624, -0.0056],
        [-0.3316,  0.0025,  0.0839,  ...,  0.0930,  1.3621, -0.0041]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [18]:
x_embedding

tensor([[-3.3180e-01,  2.5732e-03,  8.2826e-02,  ...,  9.4576e-02,
          1.3626e+00, -5.8808e-03],
        [-3.3165e-01,  2.5626e-03,  8.3458e-02,  ...,  9.3974e-02,
          1.3623e+00, -5.3152e-03],
        [-3.3142e-01,  2.4175e-03,  8.6975e-02,  ...,  8.9209e-02,
          1.3613e+00, -1.2148e-03],
        ...,
        [-3.3170e-01,  2.5562e-03,  8.3756e-02,  ...,  9.3563e-02,
          1.3622e+00, -4.8248e-03],
        [-3.3160e-01,  2.5385e-03,  8.3855e-02,  ...,  9.2990e-02,
          1.3621e+00, -4.0703e-03],
        [-3.3158e-01,  2.5564e-03,  8.3198e-02,  ...,  9.4223e-02,
          1.3624e+00, -5.5504e-03]], device='cuda:0', grad_fn=<AddmmBackward>)

In [19]:
x_delta_hat

tensor([[-4.3213e-07,  4.1455e-05,  4.5815e-05,  ...,  8.3422e-05,
          4.3456e-05, -9.3070e-05],
        [-1.7378e-05,  3.6091e-05,  5.4706e-05,  ...,  7.2762e-05,
          6.0699e-05, -1.0880e-04],
        [-4.0200e-05,  2.8171e-05,  8.8163e-05,  ...,  3.4830e-05,
          1.1909e-04, -1.8349e-04],
        ...,
        [-1.6153e-05,  3.6418e-05,  5.4499e-05,  ...,  6.8175e-05,
          6.3243e-05, -1.1674e-04],
        [-2.9553e-05,  2.7195e-05,  6.4267e-05,  ...,  5.7045e-05,
          7.7062e-05, -1.2952e-04],
        [-1.1958e-05,  3.7484e-05,  5.3717e-05,  ...,  7.5778e-05,
          5.9289e-05, -1.0814e-04]], device='cuda:0', grad_fn=<AddmmBackward>)